In [ ]:
#=============================================================================
# This is the code to run a demo of QuSing, to accompany the paper he paper
# Teaching Qubits to Sing: Mission Impossible?, by Eduardo Reck Miranda and 
# Brian N. Siegelwax.
#
# Disclaimer: this is not a software release, but a demonstration of the system 
# discussed in the said paper. You run it at your own risk. The code has not been 
# fully debugged. It was tested with the input MIDI file examples provided. 
#  
# Knowledge of Python and Qiskit is expected to run this. 
# 
# This is an outcome of the QuTune Project, developed at the Interdisciplinary 
# Centre for  Computer Music Research (ICCMR), University of Plymouth, UK.
# https://iccmr-quantum.github.io/ 
# July 08, 2022
##=============================================================================
# IMPORTS - See dependencies in README.md

import time
import matplotlib as mpl
import matplotlib.pyplot as plt

# These are Python programs to convert a MIDI file to bespoke codes for QuSing 
# and vice-versa.
import midi2code as m2c
import code2midi as c2m

# We need these to generate transition matrices.
from math import ceil, log2, sqrt

# We use this to pseudo-randomly select a known good starting sequence.
from random import randint 

# We use these for the quantum components of the algorithm. 
from qiskit import Aer, QuantumCircuit, transpile, execute, IBMQ 

# We use this to monitor queues for IBM Quantum real devices.
from qiskit.tools.monitor import job_monitor

# We use this to find out which IBM Quantum backend has the shortest queue.
from qiskit.providers.ibmq import least_busy

# We need these to optimize circuits with t|ket.
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit, IBMQBackend 

# We use the below for optimisation of quantum circuits
# https://github.com/Qiskit-Partners/mapomatic
import numpy as np
from qiskit import *
import mapomatic as mm

# https://github.com/Qiskit-Partners/mthree
import mthree

print("Imports done.")

In [ ]:
# This allows use of IBM Quantum backends if you have an IBM Quantum account.
# If you have a new IBM Quantum account, you may need to run this once with your 
# API token.
# 
#IBMQ.save_account('PASTE YOUR TOKEN HERE')
#provider = IBMQ.load_account() 

In [ ]:
#---------------------------------------------------------------------------------------
# Identify the unique notes within the training set and assign keys
#---------------------------------------------------------------------------------------
def identify_unique_notes(training_set): 
    unique_notes = {} # Create the empty dictionary.
    key = 0 # Start assigning unique keys from zero.
    # The values are strings with excess formatting, so for each value...
    for i in range(0, len(training_set)): 
        # ...extract the binary string and...
        training_set[i] = training_set[i][0:9]
        # ..if the note does not exist in the dictionary of unique notes...
        if training_set[i] not in unique_notes:
            # ...append the note to the dictionary of unique notes with its new key
            unique_notes[training_set[i]] = bin(key)[2:] 
            key += 1 # Increment the key for the next unique note.  
    # Uncomment to display the edited training set.
    #print("CLEANED TRAINING SET: " + str(training_set) + "\n") 
    return unique_notes

In [ ]:
#---------------------------------------------------------------------------------------
#  # Compress the 9-bit strings
#---------------------------------------------------------------------------------------
def convert_notes_to_keys(training_set, unique_notes, qubits):
    key_set = [] # Create the empty list.
    for i in range(0, len(training_set)): # For each note in the training set...
        key_set.append([]) # ...append an empty list to the key set and...
        key_set[i] = unique_notes[training_set[i]].zfill(qubits) # ...assign the key value of that note.
        # Uncomment to display the training and key sets together.
        #print(str(training_set[i]) + ' ' + key_set[i]) 
    return key_set

In [ ]:
#------------------------------------------------------------------------------------------- 
# Identify the unique sequences within the training set.
#------------------------------------------------------------------------------------------- 
def identify_unique_sequences(key_set, sequence_length): 
    unique_sequences = [] # Create the empty list.
    for i in range(0, (len(key_set) - sequence_length + 1)): # For each note in the key set...
        sequence = "" # ...start with nothing and...
        for j in range(0, sequence_length): # ...for the desire number of notes...
            sequence += key_set[i+j] # ...concatenate the notes.
        if sequence not in unique_sequences: # If the sequence has not already been found...
            unique_sequences.append(sequence) # ...append it to the list.
    return unique_sequences

In [ ]:
#---------------------------------------------------------------------------------------
# Create a transition matrix of zeroes
#---------------------------------------------------------------------------------------
def generate_matrix_of_zeroes(unique_sequences, unique_notes, qubits): 
    transition_matrix = {} # Create the empty dictionary.
    for i in range(0, len(unique_sequences)): # For each unique sequence...
        # ...add a row with a valid number of columns.
        transition_matrix[unique_sequences[i]] = [0] * 2**qubits 
    #print("TRANSITION MATRIX 0s: " + str(transition_matrix) + "\n")
    return transition_matrix

In [ ]:
#---------------------------------------------------------------------------------------
# Update the transition matrix with the probabilities of generating each next event 
# based on the previous n events.
#---------------------------------------------------------------------------------------
def update_matrix_with_probabilities(transition_matrix, key_set, sequence_length): 
    for i in range(0, (len(key_set) - sequence_length)): # For each note in the key set...
        sequence = "" # ...start with nothing and...
        for j in range(0, sequence_length): # ...for the desired number of notes...
            sequence += key_set[i+j] # ...concatenate the notes.
            # Look up the sequence in the transition matrix and 
            # increment the column representing the note following the sequence.
        transition_matrix[sequence][int(key_set[i+sequence_length],2)] += 1
    return transition_matrix

In [ ]:
#---------------------------------------------------------------------------------------
# Find the row which has probabilities summing to zero.
#---------------------------------------------------------------------------------------
def find_zero(transition_matrix, unique_notes): 
    for row in transition_matrix.items(): # For each row in the transition matrix...
        sum_probabilities = sum(row[1]) # ...sum the probabilities.
        if sum_probabilities == 0: # If the sum is zero...
            return row[0] # ...identify that row.

In [ ]:
#---------------------------------------------------------------------------------------
# Convert probabilities to amplitudes
#---------------------------------------------------------------------------------------
def convert_probabilities_to_amplitudes(transition_matrix, unique_notes): 
    for row in transition_matrix.items(): # For each row in the transition matrix...
        # ...preserve the sum of the probabilities before changing the values.
        sum_probabilities = sum(row[1]) 
        if sum_probabilities == 0: # If the sum of the probabilities is zero...
            transition_matrix.pop(row[0]) # ...remove the row...
        else: # ...otherwise...
            for j in range(0,len(unique_notes)): # ...for each column...
                # ...update each value with the square root of its normalized value.
                row[1][j] = sqrt(row[1][j]/sum_probabilities) 
    return transition_matrix

In [ ]:
#---------------------------------------------------------------------------------------
# Select a backend
#---------------------------------------------------------------------------------------
def select_backend(backend, qubits): # Select a backend.
    if backend == "Aer": # If the simulator has been designated...
        backend = Aer.get_backend('aer_simulator') # ...use the simulator.
    else: # For real hardware:
        # If we need more than 5 qubits, we need to use ibmq_jakarta.
        if qubits > 5: 
            # Access backends with more than 5 available qubits.
            provider = IBMQ.get_provider(hub='ibm-q-research') 
            # ibmq_jakarta is an available 7-qubit backend.
            backend = provider.get_backend('ibmq_jakarta') 
            #backend = provider.get_backend('ibmq_guadalupe') 
        else: # Select either the least_busy or a specific backend.
            provider = IBMQ.get_provider(hub='ibm-q') 
            # Access freely-available backends and...
            # ...get the specifically-named backend.
            if backend == "least_busy":
                backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 and not x.configuration().simulator and x.status().operational==True))
            else:
                backend = provider.get_backend(backend) 
    return backend

In [ ]:
#---------------------------------------------------------------------------------------
# Generate the next event
#---------------------------------------------------------------------------------------
def generate_note(transition_matrix, qubits, dictionary_key, backend, unique_notes, shots): 
    # The initial state is the amplitudes of the appropriate row.
    initial_state = transition_matrix[dictionary_key] 
    #
    #---------
    # Iterate through the amplitudes and...
    # If theres is an amplitude = 1.0, then there is no need to build circuit: 
    # this has to be the next note
    for i in range(0,len(initial_state)): 
        # ...if one of them is 1.0...
        if initial_state[i] == 1.0:
            # ...that note has to be our next note.
            next_note = list(unique_notes.values())[i].zfill(qubits)  
            skip = True
            backend = "" 
            #print("AMP=1.0 => NO CIRCUIT IS NECESSARY ...")
            return next_note, skip, backend   
    skip = False
    # Select a backend. For each note we select a backend.
    backend = select_backend(backend, qubits) 
    #print("BACKEND FOR THIS NOTE: " + str(backend))
           
    #---------
    #---------    
    # Dynamically build the quantum circuit and print it
    qc = QuantumCircuit(qubits, qubits) 
    #
    # Assign the qubits as a list so that the qubits can be assigned dynamically.
    assignment = [] 
    for i in range(0, qubits): # For each qubit...
        assignment.append(i) # ...assign it in order from 0.
    # Encode the amplitudes.    
    qc.initialize(initial_state, assignment) 
    # Measure all the qubits.
    qc.measure_all() 
    
    # Uncomment the below to write initial (non-optimised) QASM code to a file.
    # Give the circuit a filename based on the transition_matrix key...  
    #---------
    #qc_initial = transpile(qc, backend=backend, seed_transpiler=11, optimization_level=3, layout_method='sabre', routing_method='sabre', approximation_degree=0) # Optimize with circuit with Qiskit Pass Manager. 
    #qasm_file = "qasm" + dictionary_key + "_initial.txt" 
    #f = open(qasm_file, 'w') # ...then create/open the file...
    #f.write(qc_initial.qasm()) # ...and write the OpenQASM of the circuit to the file.
    #f.close()
    
    #---------
    # If we're using the simulator, does not get optimized
    if str(backend) == 'aer_simulator': 
        qc = transpile(qc, backend=backend, seed_transpiler=11, optimization_level=3, layout_method='sabre', routing_method='sabre', approximation_degree=0) # Optimize with circuit with Qiskit Pass Manager.          
    # If using a hardware backend, optimize the circuit with a number of tools.    
    else:  
        # t|ket
        if str(backend) == 'ibmq_jakarta': # If we're using ibmq_jakarta...
            provider = IBMQ.get_provider(hub='ibm-q-research') # ...we need its provider attributes.
        else:
            provider = IBMQ.get_provider(hub='ibm-q')
        qc = qc.decompose().decompose() # Undo initialize() abstractions.
        tk_qc = qiskit_to_tk(qc) # Convert the Qiskit circuit to a t|ket circuit.
        tk_backend = IBMQBackend(hub=str(provider.credentials.hub), group=str(provider.credentials.group), project=str(provider.credentials.project), backend_name=str(backend)) # Select our already-selected backend.
        tk_backend.default_compilation_pass(2).apply(tk_qc) # Optimize the t|ket circuit.
        qc = tk_to_qiskit(tk_qc) # Convert the t|ket circuit back into a Qiskit circuit.
        
        # Qiskit Pass Manager
        qc = qc.decompose().decompose()
        trans_qc_list = transpile([qc]*20, provider.get_backend(str(backend)), seed_transpiler=11, optimization_level=3, layout_method='sabre', routing_method='sabre', approximation_degree=0)
        best_cx_count = [circ.count_ops()['cx'] for circ in trans_qc_list]
        best_idx = np.where(best_cx_count == np.min(best_cx_count))[0][0]
        qc = trans_qc_list[best_idx]

        # https://github.com/Qiskit-Partners/mapomatic
        small_qc = mm.deflate_circuit(qc)
        layouts = mm.matching_layouts(small_qc, backend)
        scores = mm.evaluate_layouts(small_qc, layouts, backend)
        qc = transpile(small_qc, backend, initial_layout=scores[0][0])
     
        # Uncomment the below to write the optimized QASM to a file.
        # Give the circuit a filename based on the transition_matrix key...
        #qasm_file = "qasm" + dictionary_key + "_optimized.txt" 
        #f = open(qasm_file, 'w') # ...then create/open the file...
        #f.write(qc.qasm()) # ...and write the OpenQASM of the circuit to the file.
        #f.close()

    ### Print the circuits
    #print("ORIGINAL CIRCUIT: ...")
    #print(qc_initial)
    
    #print("\n==================================\n")
    #print("OPTIMISED CIRCUIT: ...")
    #print(qc)    


    job = execute(qc, backend, shots=shots) # Generate only one next note.
    if str(backend) == "aer_simulator": # If we're using the simulator...
        pass # ...we don't need to use the job monitor...
    else: # ...otherwise...
        job_monitor(job) # ...use the job monitor to display our wait in the queue.
    result = job.result() # Retrieve the result.
    
    if str(backend) == "aer_simulator":
        counts = result.get_counts()
        counts = str(max(counts, key=counts.get)) # In case of shots > 1, find the result with the highest count.
        next_note = counts[0:qubits]
    else:
        # https://github.com/Qiskit-Partners/mthree
        mit = mthree.M3Mitigation(backend)
        mit.cals_from_system(qubits=assignment, shots=shots)
        raw_counts = result.get_counts(qc)
        m3_quasi = mit.apply_correction(raw_counts, assignment)
        # In case of shots > 1, find the result with the highest count.
        counts = str(max(m3_quasi, key=m3_quasi.get)) 
        next_note = counts[0:qubits]
    
    return next_note, skip, backend

In [ ]:
#---------------------------------------------------------------------------------------
# Convert keys back to notes
#---------------------------------------------------------------------------------------
def convert_keys_to_notes(new_film_music, unique_notes, qubits): 
    # For each generated note...
    for i in range(0, len(new_film_music)): 
        # ...iterate through the dictionary of unique notes...
        for key, value in unique_notes.items(): 
            # ...if there's a match...
            if new_film_music[i] == value.zfill(qubits): 
                # ...replace the key with its original 9-bit string.
                new_film_music[i] = key 
    return new_film_music

In [ ]:
#-----------------------------------------------------------------------------------------
# Calculate the difference between the training set and the new set
#-----------------------------------------------------------------------------------------
def calculate_difference(training_set, new_film_music): 
    difference = 0 # Start at zero.
    # If the training set is longer...
    if len(training_set) > len(new_film_music):
        # ...start with the new set...
        shortest = len(new_film_music) 
    else: # ...otherwise...
        # ...start with the training set.
        # For the length of the shorter set...
        shortest = len(training_set) 
    for i in range(0,shortest): 
        # ...add the absolute value of the difference of the values. 
        difference += abs(int(training_set[i], 2) - int(new_film_music[i], 2)) 
    return difference

In [ ]:
#-----------------------------------------------------------------------------------------
# Save the new set to a raw file
#-----------------------------------------------------------------------------------------
def save_new_film_music(new_film_music, original_filename, sequence_length, backend, number_of_new_notes, shots, original, use_initial): # Save the new set to a file.
    filename = original_filename + " " + str(sequence_length) + "n " + str(backend) + " " \
    + str(number_of_new_notes) + "notes " + str(shots) + "shots " + str(original) + " " \
    + str(use_initial) + " " + str(time.time()) + ".txt"
    f = open(filename, 'w') # Open a file in write mode.
    for i in range(len(new_film_music)): # For each new note...
        f.write(new_film_music[i]) # ...append the note to the file and...
        if i < (len(new_film_music) - 1): # ...if it's not the last note...
            f.write("\n") # ...append a line break.
    print("\nFile saved as: " + str(filename) + "\n")
    f.close() # Close the file.

In [ ]:
#-----------------------------------------------------------------------------------------
# To Run the process of learning and composing a new piece
#-----------------------------------------------------------------------------------------
def make_film_music(filename, sequence_length, backend, number_of_new_notes, shots, original, use_initial):
    # Uncomment the print commands below to follow what the system is doing.
    training_set, pitch_dictionary, duration_dictionary = m2c.convert_MIDI_to_code(filename)
    #print("CONVERTED TRAINING SET: " + str(training_set) + "\n") 
    #
    # Identify the unique notes within the training set and assign keys.
    unique_notes = identify_unique_notes(training_set) 
    print("UNIQUE EVENTS: " + str(unique_notes) + "\n") # Display the list of unique notes.
    print("COUNT OF EVENTS: " + str(len(unique_notes)) + "\n") # Display the count of unique notes.
    #
    # Calculate the number of qubits we need.
    qubits = ceil(log2(len(unique_notes))) 
    print("NUMBER OF REQUIRED QUBITS: " + str(qubits) + "\n")
    #
    # Compress the 9-bit strings.
    key_set = convert_notes_to_keys(training_set, unique_notes, qubits) 
    #print("COMPRESSED CODES: " + str(key_set) + "\n") 
    #
    # Identify the unique sequences within the training set.
    unique_sequences = identify_unique_sequences(key_set, sequence_length) 
    #print("UNIQUE SEQUENCES: " + str(unique_sequences) + "\n")
    #
    # Create a transition matrix of zeroes.
    transition_matrix = generate_matrix_of_zeroes(unique_sequences, unique_notes, qubits) 
    #print("TRANSITION MATRIX 0s: " + str(transition_matrix) + "\n")
    #
    # Update the transition matrix with the probabilities of generating each next note 
    # based on the previous n notes.
    transition_matrix = update_matrix_with_probabilities(transition_matrix, key_set, sequence_length) 
    #print("TRANSITION MATRIX WITH COUNTS: " + str(transition_matrix) + "\n")
    #
    # Find the row which has probabilities summing to zero.
    zero = find_zero(transition_matrix, unique_notes)
    #print("ROW SUMMING TO ZERO - TO REMOVE: " + str(zero) + "\n")
    #
    # Remove the row that has probabilities summing to zero.
    if zero != None:
        transition_matrix.pop(zero)
        
    # Convert probabilities to amplitudes.
    transition_matrix = convert_probabilities_to_amplitudes(transition_matrix, unique_notes) 
    #print("TRANSITION MATRIX WITH AMPLITUDES: " + str(transition_matrix) + "\n")
    #
    # Define backend
    backend_parameter = backend # Makes a copy of the backend parameter passed to make_film_music().
    print("BACKEND: " + str(backend_parameter) + "\n") # Display the selected backend.
    #
    # Create an empty list for the newly-generated notes.
    new_film_music = [] 

    if original == True:
        # Start with the same sequence as the original tune.
        sequence = unique_sequences[0] 
    else:
        # Randomly select a known good initial sequence.
        sequence = unique_sequences[randint(0,(len (unique_sequences) - 1))] 
        
    if use_initial == True:
        for i in range(0, sequence_length):
            classical_note = sequence[(0 + (qubits * i)) : ((0 + (qubits * i)) + qubits)]
            print(str(len(new_film_music)+1) + "/" + str(number_of_new_notes) + ": " + str(classical_note) + " initial sequence")
            new_film_music.append(classical_note)
    
    print("INITIAL STARTING SEQ: " + str(sequence) + "\n")
    
    #----------------------------------------------------------
    # Loop until we've generated the desired number of notes. 
    good_measurements = 0
    noisy_measurements = 0
    skipped_measurements = 0
    noise = False
    print("\n=============================")
    print("START GENERATIVE PROCESS")
    print("==============================\n")    
    while len(new_film_music) < (number_of_new_notes): 
        #
        #----------------
        # Remove dead end notes that cause problems.
        # THIS WILL NEED TO BE AUTOMATIC TO DEAL WITH OTHER TRAINING SETS
        #
        if ((filename == 'Bach' and sequence.endswith('1000') == True) \
            or ((filename == 'MissionImpossible') and sequence.endswith('10110') == True)): 
            # ...so, disregard them...
            sequence = unique_sequences[randint(0,(len (unique_sequences) - 1))] 
            continue # ...and try again.          
        #----------------
        #        
        # Generate the next note.
        next_note = generate_note(transition_matrix, qubits, sequence, backend_parameter, unique_notes, shots)
        print("PREVIOUS EVENT - DICTIONARY KEY:" + str(sequence))
        print("GENERATED EVENT: " + str(next_note[0])) # Display the selected backend.
        #
        # Determine what the next n-note sequence would be.
        #new_sequence = sequence[qubits:(qubits * sequence_length)] + next_note
        new_sequence = sequence[qubits:(qubits * sequence_length)] + next_note[0]
        #
        # If the new sequence is not valid...
        if new_sequence not in unique_sequences:           
            noise = True          
            continue # ...try again..
        else: # ...otherwise...
            
            skip = next_note[1]
            if (noise == False) & (skip == False):
                good_measurements += 1
            if (noise == True) & (skip == False):
                noisy_measurements += 1
                noise = False
            if skip == True:
                skipped_measurements += 1
                skip = False  
            sequence = new_sequence # ...use the new sequence for the next iteration and...
            new_film_music.append(next_note[0]) # ...save the newly-generated note.
            # Display our progress.
            print("PROGRESS: " + str(len(new_film_music)) + "/" + str(number_of_new_notes) + ": " + str(next_note[0]) + " " + str(next_note[2]) + "\n") 
    print("=============================\n")
    #----------------------------------------------------------
    #
    # Convert keys back to notes.
    #print("NEW SEQUENCE OF KEYS: " + str(new_film_music) + "\n")
    new_film_music = convert_keys_to_notes(new_film_music, unique_notes, qubits) 
    #print("NEW COMPOSITION: " + str(new_film_music) + "\n")
    #
    # Calculate the difference between the training set and the new set.
    #print("DIFFERENCE: " + str(calculate_difference(training_set, new_film_music)) + "\n") 
    #
    # Convert notes to MIDI and save MIDI file
    c2m.convert_code_to_MIDI(new_film_music, pitch_dictionary, duration_dictionary, sequence_length, backend_parameter, number_of_new_notes, shots)
    
    return good_measurements, noisy_measurements, skipped_measurements

print("Functions definitions done.")

In [ ]:
#-----------------------------------------------------------------------------------------
# These are QuTune settings, wich can be modified. See paper.
#-----------------------------------------------------------------------------------------
# The name of the MIDI file with the original tune. (Don't put the extension .mid.)
#filename = 'Bach' 
filename = 'MissionImpossible'

# Number of previous events to consider to generate each new event; tested with 1, 2 and 3
sequence_length = 3 

# The number of events to generate for the new composition. Warning: not too many if using 
# hardware backend. 
new_notes = 24 

# Number of shots per run.
shots = 1 

# If True, start with the same event as the original tune, otherwise start with a 
# random-but-known-good event.
original = False 

# If True, include the initial event in the new tune, otherwise start with the first 
# generated tune.
use_initial = True 

In [ ]:
#-----------------------------------------------------------------------------------------
# Run on Aer simulator
#-----------------------------------------------------------------------------------------
new_film_music = make_film_music(filename, sequence_length, 'Aer', new_notes, shots, original, use_initial)
sim_good_measurements = new_film_music[0]
sim_noisy_measurements = new_film_music[1]
sim_skipped_measurements = new_film_music[2]

In [ ]:
#-----------------------------------------------------------------------------------------
# Run on hardware backend. Don't forget to provide your IBM Quantum token above.
# Backends other than ibmq_lima are possible.
#-----------------------------------------------------------------------------------------
new_film_music = make_film_music(filename, sequence_length, 'ibmq_lima', new_notes, shots, original, use_initial)
real_good_measurements = new_film_music[0]
real_noisy_measurements = new_film_music[1]
real_skipped_measurements = new_film_music[2]

In [ ]:
#-----------------------------------------------------------------------------------------
# These were used to generate the plots in the paper, Figs, 17, 18, 19, etc.
#-----------------------------------------------------------------------------------------
print("real good: " + str(real_good_measurements) + ", noisy: " + str(real_noisy_measurements) + ", skipped: " + str(real_skipped_measurements))
print("sim good: " + str(sim_good_measurements) + ", noisy: " + str(sim_noisy_measurements) + ", skipped: " + str(sim_skipped_measurements))

import numpy as np
import matplotlib.pyplot as plt

groups = 3
real = (real_good_measurements, real_noisy_measurements, real_skipped_measurements)
sim = (sim_good_measurements, sim_noisy_measurements, sim_skipped_measurements)

fig, ax = plt.subplots()
index = np.arange(groups)
bar_width = 0.35
opacity = 0.8

rects1 = plt.bar(index, real, bar_width,
alpha=opacity,
color='b',
label='real')

rects2 = plt.bar(index + bar_width, sim, bar_width,
alpha=opacity,
color='g',
label='sim')

plt.xlabel('Possible Measurement Outcomes')
plt.ylabel('Number of Runs')
plt.title('Real vs Sim')
plt.xticks(index + (bar_width/2), ('good', 'noisy', 'skipped'))
plt.legend()

plt.tight_layout()
plt.show()